In [1]:
!pip uninstall -y  google-ai-generativelanguage

Found existing installation: google-ai-generativelanguage 0.7.0
Uninstalling google-ai-generativelanguage-0.7.0:
  Successfully uninstalled google-ai-generativelanguage-0.7.0


In [3]:
!pip install langchain langchain-community langchain-google-genai # required libraries

In [4]:
!pip install gradio pypdf python-docx docx2txt faiss-cpu # document libraries

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 62.4 MB/s eta 0:00:00


In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "PASTE YOUR OWN API KEY" # environment setting up with API

In [6]:
# Importing required libraries
from typing import List, Union
import tempfile, os

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader

from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

import gradio as gr

In [7]:
# Model requirement
GENERATION_MODEL = "models/gemini-2.5-flash"
EMBEDDING_MODEL = "models/text-embedding-004"

llm = ChatGoogleGenerativeAI(model=GENERATION_MODEL, temperature=0.7)
embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)

In [8]:
# Blog generation prompt
blog_prompt = PromptTemplate(
    input_variables=["topic", "tone", "length"],
    template=(
        "You are an expert content writer. Write a {length} blog post on '{topic}' "
        "in a {tone} tone. Use clear headings and bullet points where helpful."
    ),
)
blog_chain = blog_prompt | llm

In [9]:
# Copywriting prompt
copy_prompt = PromptTemplate(
    input_variables=["goal", "audience", "style"],
    template=(
        "Create compelling marketing copy to achieve the goal: {goal}. "
        "Target audience: {audience}. Style: {style}. Provide 2–3 variants."
    ),
)
copy_chain = copy_prompt | llm

In [10]:
# Rewriting prompt
rewrite_prompt = PromptTemplate(
    input_variables=["text", "tone"],
    template=("Rewrite the following text in a {tone} tone while preserving meaning:\n\n{text}")
)
rewrite_chain = rewrite_prompt | llm

In [11]:
# Summarizer prompt
short_sum_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text concisely in 3–5 bullets:\n\n{text}"
)
short_sum_chain = short_sum_prompt | llm

In [12]:
# Chatbot prompt
chat_memory = ConversationBufferMemory(return_messages=True)
# A lightweight conversation template:
chat_prompt = PromptTemplate(
    input_variables=["history", "user_input"],
    template=(
        "You are a helpful, concise assistant.\n"
        "Conversation so far:\n{history}\n\n"
        "User: {user_input}\nAssistant:"
    ),
)

def chat_infer(user_input: str):
    # build a simple few-shot style message with memory
    history_text = ""
    for m in chat_memory.chat_memory.messages:
        role = "User" if m.type == "human" else "Assistant"
        history_text += f"{role}: {m.content}\n"
    filled = chat_prompt.format(history=history_text.strip(), user_input=user_input)
    resp = llm.invoke(filled)
    # update memory
    chat_memory.chat_memory.add_user_message(user_input)
    chat_memory.chat_memory.add_ai_message(resp.content)
    return resp.content

def reset_chat():
    chat_memory.clear()
    return "Chat memory cleared."


/tmp/ipython-input-1705632888.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationBufferMemory(return_messages=True)


In [13]:
# -------- Document loading --------
def load_docs(file_path: str):
    ext = file_path.lower()
    if ext.endswith(".pdf"):
        return PyPDFLoader(file_path).load()
    if ext.endswith(".docx"):
        return Docx2txtLoader(file_path).load()
    if ext.endswith(".txt"):
        return TextLoader(file_path, encoding="utf-8").load()
    raise ValueError("Unsupported file type. Upload PDF, DOCX, or TXT.")

# -------- Chunking --------
def split_docs(docs, chunk_size=1200, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, separators=["\n\n", "\n", " ", ""]
    )
    return splitter.split_documents(docs)

# -------- Summarization (map-reduce over long docs) --------
def summarize_long_docs(docs):
    # docs should be a list[Document]; chunk for long inputs
    chunks = split_docs(docs)
    chain = load_summarize_chain(llm, chain_type="map_reduce")  # map (per chunk) + reduce
    return chain.run(chunks)

# -------- Build a retriever for QA (RAG) --------
def build_retriever_from_docs(docs):
    chunks = split_docs(docs)
    vs = FAISS.from_documents(chunks, embeddings)
    return vs.as_retriever(search_kwargs={"k": 4})

def qa_over_docs(question: str, retriever):
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=False
    )
    return chain.run(question)


In [16]:
# Loading required libraries for UI
import gradio as gr
import os, tempfile
from typing import List
import sys

from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA

# === BLOG GENERATOR ===
def generate_blog(topic, tone, length):
    return blog_chain.invoke({"topic": topic, "tone": tone, "length": length}).content

# === COPYWRITER ===
def generate_copy(goal, audience, style):
    return copy_chain.invoke({"goal": goal, "audience": audience, "style": style}).content

# === REWRITER ===
def rewrite_text(text, tone):
    return rewrite_chain.invoke({"text": text, "tone": tone}).content

# === SUMMARIZER ===
def summarize_text(text):
    return short_sum_chain.invoke({"text": text}).content

# === CHAT ===
def chat_bot(user_input):
    return chat_infer(user_input)

def reset_memory():
    return reset_chat()

# === DOC Q&A ===
retriever = None

def upload_docs(file):
    global retriever
    docs = load_docs(file.name)
    retriever = build_retriever_from_docs(docs)
    return "Document uploaded & indexed!"

def ask_doc(question):
    if retriever is None:
        return "Please upload a document first!"
    return qa_over_docs(question, retriever)

# === UI DESIGN ===
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## ISOTOPE AI - Where AI meets Creativity")

    with gr.Tab("📝 Blog Generator"):
        topic = gr.Textbox(label="Topic")
        tone = gr.Dropdown(["Professional", "Casual", "Friendly", "Persuasive"], label="Tone")
        length = gr.Radio(["Short", "Medium", "Long"], label="Length")
        blog_btn = gr.Button("Generate Blog")
        blog_output = gr.Textbox(label="Generated Blog", lines=15)
        blog_btn.click(generate_blog, [topic, tone, length], blog_output)

    with gr.Tab("🎯 Copywriting"):
        goal = gr.Textbox(label="Goal")
        audience = gr.Textbox(label="Audience")
        style = gr.Dropdown(["Formal", "Casual", "Creative"], label="Style")
        copy_btn = gr.Button("Generate Copy")
        copy_output = gr.Textbox(label="Generated Copy", lines=10)
        copy_btn.click(generate_copy, [goal, audience, style], copy_output)

    with gr.Tab("✍️ Rewrite"):
        rewrite_in = gr.Textbox(label="Input Text", lines=8)
        rewrite_tone = gr.Dropdown(["Professional", "Casual", "Friendly"], label="Tone")
        rewrite_btn = gr.Button("Rewrite")
        rewrite_out = gr.Textbox(label="Rewritten Text", lines=8)
        rewrite_btn.click(rewrite_text, [rewrite_in, rewrite_tone], rewrite_out)

    with gr.Tab("📌 Summarizer"):
        sum_in = gr.Textbox(label="Text to Summarize", lines=10)
        sum_btn = gr.Button("Summarize")
        sum_out = gr.Textbox(label="Summary", lines=5)
        sum_btn.click(summarize_text, sum_in, sum_out)

    with gr.Tab("💬 Chatbot"):
      chatbot_ui = gr.Chatbot(label="Chat with Gemini")
      chat_in = gr.Textbox(label="Your Message", placeholder="Type here...")

      with gr.Row():
        chat_btn = gr.Button("Send")
        reset_btn = gr.Button("Reset Chat")

    # === Functions ===
      def chat_flow(user_input, history):
        # call LangChain + memory
        reply = chat_infer(user_input)
        history = history + [(user_input, reply)]
        return history, ""

      def reset_flow():
        reset_chat()  # clears LangChain memory
        return [], ""

    # === Wiring ===
      chat_btn.click(chat_flow, [chat_in, chatbot_ui], [chatbot_ui, chat_in])
      reset_btn.click(reset_flow, outputs=[chatbot_ui, chat_in])


    with gr.Tab("📂 Document Q&A"):
        doc_file = gr.File(label="Upload Document (PDF/DOCX/TXT)")
        doc_status = gr.Textbox(label="Status")
        doc_file.upload(upload_docs, doc_file, doc_status)

        doc_q = gr.Textbox(label="Ask a Question")
        doc_btn = gr.Button("Ask")
        doc_a = gr.Textbox(label="Answer", lines=6)
        doc_btn.click(ask_doc, doc_q, doc_a)

# 🚀 Launch with a public tunnel (Colab → Replit/API use)
demo.launch(share=True)

/tmp/ipython-input-592775175.py:84: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(label="Chat with Gemini")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4e1528b52e81d5247a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
